In [1]:
from __future__ import annotations

In [2]:
import pandas as pd
from pathlib import Path
from pydantic import BaseModel
import csv
from collections import Counter, defaultdict
from typing import Optional
import yaml
import enum

In [3]:
p = Path('../data')

In [5]:
def parse_matches(file_path: str) -> list[str]:
    matches = []
    for path in Path(file_path).iterdir():
        match = path.parts[-1]
        matches.append(match)
    return matches

In [6]:
def parse_maps(file_path: str):
    maps = []
    for path in Path(file_path).iterdir():
        map = path.parts[-1]
        maps.append(map)
    return maps

In [7]:
def get_round_counts(file_path: str) -> dict[str, int]:
    rounds = []
    round_path = Path(file_path).joinpath("time_series.csv")
    with open(round_path, 'r') as f:
        csvreader = csv.DictReader(f)
        for row in csvreader:
            rounds.append(row["ResultType"])
    round_counts = Counter(rounds)
    return dict(round_counts)

In [8]:
def get_map_info(file_path: str) -> dict[str, str|int]:
    map_path = Path(file_path).joinpath("map_info.yaml")
    with open(map_path, 'r') as f:
        map_data = yaml.load(f, Loader=yaml.FullLoader)
        return map_data

In [9]:
class Team(BaseModel):
    name: str
    score: int

class Eco(enum.Enum):
    NONE = "-"
    LIGHT = "$"
    MED = "$$"
    FULL = "$$$"

class Round(BaseModel):
    number: int
    winner: Optional[str]
    side_winner: str
    result_type: str
    team1_eco: Eco
    team2_eco: Eco

    @property
    def is_pistol(self) -> bool:
        return True if self.number == 1 or self.number == 13 else False

class Results(BaseModel):
    elim: int = 0
    boom: int = 0
    defuse: int = 0

class Map(BaseModel):
    name: str
    duration: str
    team1: Team
    team2: Team
    round_results: Results
    rounds: list[Round]

    # these need refactor to Map methods
    # will use self.rounds, self.team1, self.team2
    def set_round_winner_name(self):
        ct_to_t = 0
        for r in self.rounds:
            if r.number < 13 and r.side_winner == "ct":
                ct_to_t += 1
            elif 13 <= r.number and r.side_winner == "t":
                ct_to_t += 1
        if ct_to_t == self.team1.score:
            # team1 was ct first
            for r in self.rounds:
                if r.number < 13:
                    if r.side_winner == "ct":
                        r.winner = self.team1.name
                    else:
                        r.winner = self.team2.name
                elif 13 <= r.number:
                    if r.side_winner == "t":
                        r.winner = self.team1.name
                    else:
                        r.winner = self.team2.name   
        else:
            # team 1 was t first
            for r in self.rounds:
                if r.number < 13:
                    if r.side_winner == "t":
                        r.winner = self.team1.name
                    else:
                        r.winner = self.team2.name
                elif 13 <= r.number:
                    if r.side_winner == "ct":
                        r.winner = self.team1.name
                    else:
                        r.winner = self.team2.name
        return


    def pistol_wins(self, team_name: str) -> int:
        return len([r for r in self.rounds if r.is_pistol and r.winner == team_name])


    def bonus_wins(self, team_name: str) -> int:
        # specifically looks at rounds after pistol wins
        pistol_win_rounds = [r for r in self.rounds if r.is_pistol and r.winner == team_name]
        bonus_wins = 0
        for pistol in pistol_win_rounds:
            bonus_round = [r for r in self.rounds if r.number == pistol.number + 1]
            if not bonus_round:
                continue # most likely 13-0
            if bonus_round[0].winner == team_name:
                bonus_wins += 1
        return bonus_wins

    def third_round_wins(self, team_name: str) -> int:
        wins = 0
        for r in self.rounds:
            if r.number == 3 or r.number == 13:
                if r.winner == team_name:
                    wins += 1
        return wins

    @property
    def winner(self) -> str:
        if self.team1.score > self.team2.score:
            return self.team1.name
        else:
            return self.team2.name

    def is_winner(self, team_name: str) -> bool:
        return self.winner == team_name

In [10]:
def get_round_data(file_path: str) -> list[Round]:
    rounds = []
    round_path = Path(file_path).joinpath("time_series.csv")
    with open(round_path, 'r') as f:
        csvreader = csv.DictReader(f)
        for row in csvreader:
            r = Round(
                number=row['RoundNumber'],
                side_winner=row['Winner'],
                result_type=row['ResultType'],
                team1_eco=Eco(row['Team1Economy']),
                team2_eco=Eco(row['Team2Economy']),
            )
            rounds.append(r)
    return rounds

In [11]:
map_data = []
for match in parse_matches("../data"):
    maps = parse_maps(f"../data/{match}/maps")
    for map_ in maps:
        round_counts = get_round_counts(f"../data/{match}/maps/{map_}")
        round_data = get_round_data(f"../data/{match}/maps/{map_}")
        map_info = get_map_info(f"../data/{match}/maps/{map_}")
        team1 = Team(
            name=map_info["Team1"]["name"],
            score=map_info["Team1"]["score"]
        )
        team2 = Team(
            name=map_info["Team2"]["name"],
            score=map_info["Team2"]["score"]
        )
        map_info = Map(
            name=map_info["Name"],
            duration=map_info["Duration"],
            team1=team1,
            team2=team2,
            round_results=Results(**round_counts),
            rounds=round_data,
        )
        map_info.set_round_winner_name()
        map_data.append(map_info)

print(map_data[0])

Map name='Haven' duration='49:37' team1=<Team name='G2 Esports' score=13> team2=<Team name='F4Q' score=8> round_results=<Results elim=13 boom=2 defuse=6> rounds=[<Round number=1 winner='F4Q' side_winner='ct' result_type='elim' team1_eco=<Ec…


In [34]:
# this can be good input for ML 😃
def round_winners_dataset(m_data: list[Map]):
    data = []
    for m in m_data:
        for team in [m.team1, m.team2]:
            row = {
                "map_name": m.name,
                "team_name": team.name,
                "won": m.is_winner(team.name),
            }
            for r in m.rounds:
                row = row | {f"round_{r.number}": r.winner == team.name}
            data.append(row)
    return data

In [39]:
data = round_winners_dataset(map_data)
df = pd.DataFrame(data)

import pickle

with open('ts-data.pkl', 'wb') as f:
    pickle.dump(df, f)

(132, 31)


In [19]:
round_winners_dataset(map_data)[0]

{'G2 Esports': 0,
 'F4Q': 0,
 'map_name': 'Haven',
 'map_winner': 'G2 Esports',
 'round_number': 0}

In [46]:
teams = set()
for m in map_data:
    teams.add(m.team1.name)
    teams.add(m.team2.name)

len(teams)

15

In [47]:
team_data = []
for team in teams:
    team_maps = [m for m in map_data if m.team1.name == team or m.team2.name == team]
    for map_ in team_maps:
        t_data = {
            "team_name": team,
            "pistol_wins": map_.pistol_wins(team),
            "bonus_wins": map_.bonus_wins(team),
            "third_round_wins": map_.third_round_wins(team),
            "is_winner": map_.is_winner(team),
        }
        team_data.append(t_data)

team_data[0]

{'team_name': 'Vision Strikers',
 'pistol_wins': 2,
 'bonus_wins': 1,
 'third_round_wins': 1,
 'is_winner': True}

In [49]:
team_df = pd.DataFrame(team_data)
team_df

,team_name,pistol_wins,bonus_wins,third_round_wins,is_winner
0,Vision Strikers,2,1,1,True
1,Vision Strikers,1,1,1,True
2,Vision Strikers,1,1,0,True
3,Vision Strikers,2,2,2,True
4,Vision Strikers,1,1,0,True
...,...,...,...,...,...
127,Sentinels,1,1,2,False
128,Sentinels,0,0,0,False
129,Sentinels,1,1,1,False
130,Sentinels,1,1,1,True


In [68]:
team_df.corr()['is_winner']

pistol_wins         0.603382
bonus_wins          0.577899
third_round_wins    0.314627
is_winner           1.000000
Name: is_winner, dtype: float64

In [69]:
info = []
for team in team_df.team_name.unique():
    temp = team_df[team_df.team_name == team]
    wins = temp.is_winner.sum()
    ratio = wins / temp.shape[0]
    pistol_ratio = temp.pistol_wins.sum() / (temp.shape[0] * 2)
    info.append([team, round(ratio, 2), round(pistol_ratio, 2), temp.shape[0]])

sorted(info, key=lambda x: x[-2], reverse=True)

[['Gambit Esports', 0.8, 0.73, 15],
 ['Vision Strikers', 0.71, 0.64, 7],
 ['Envy', 0.73, 0.59, 11],
 ['SuperMassive Blaze', 0.38, 0.56, 8],
 ['100 Thieves', 0.6, 0.55, 10],
 ['Vivo Keyd', 0.33, 0.5, 6],
 ['Sentinels', 0.5, 0.5, 12],
 ['G2 Esports', 0.64, 0.46, 14],
 ['KRÜ Esports', 0.44, 0.44, 9],
 ['Crazy Raccoon', 0.33, 0.42, 6],
 ['Acend', 0.5, 0.4, 10],
 ['F4Q', 0.2, 0.4, 10],
 ['ZETA DIVISION', 0.2, 0.4, 5],
 ['Havan Liberty', 0.0, 0.25, 4],
 ['Paper Rex', 0.2, 0.2, 5]]

In [50]:
# looks at different results for the maps
map_results = dict()

for map_name in [map.name for map in map_data]:
    all_results = []
    for x in [m for m in map_data if m.name == map_name]:
        all_results.extend(x.round_results)

    d = defaultdict(int)
    for item in all_results:
        d[item[0]] += item[1]
    map_results[map_name] = d
map_results

{'Haven': defaultdict(int, {'elim': 210, 'boom': 25, 'defuse': 44}),
 'Ascent': defaultdict(int, {'elim': 170, 'boom': 9, 'defuse': 26}),
 'Bind': defaultdict(int, {'elim': 123, 'boom': 17, 'defuse': 29}),
 'Split': defaultdict(int, {'elim': 220, 'boom': 24, 'defuse': 47}),
 'Icebox': defaultdict(int, {'elim': 211, 'boom': 18, 'defuse': 38}),
 'Breeze': defaultdict(int, {'elim': 86, 'boom': 13, 'defuse': 14})}